# 추천 시스템(Recommendation System)

1. 콘텐츠 기반 필터링(Content-based Filtering)
    - 아이템의 속성을 기반으로 사용자에게 적합한 아이템 추천

2. 협업 필터링 (Collaborative Filtering)
    - 사용자들 간의 유사성을 기반으로 추천
    - 사용자 기반과 아이템 기반으로 각각 추천할 수 있음

3. 하이브리드 추천 시스템 (Hybird Recommendation System)
    - 협업 필터링과 콘텐츠 기반 필터링을 결합하여 추천

- 영화 데이터
    1. **id**: 영화의 고유 ID를 나타냄.
    2. **title**: 영화의 제목.
    3. **budget**: 영화 제작에 소요된 예산 (단위: USD).
    4. **popularity**: 영화의 인기 점수. TMDb에서 제공하는 영화의 인기도를 나타냄.
    5. **genres**: 영화의 장르를 나타내며, 여러 장르가 포함된 경우 리스트로 표현됨.
    6. **overview**: 영화의 줄거리나 개요를 설명하는 텍스트.
    7. **release_date**: 영화의 개봉 날짜.
    8. **revenue**: 영화의 총 수익 (단위: USD).
    9. **runtime**: 영화의 상영 시간 (단위: 분).
    10. **vote_average**: TMDb에서 제공하는 영화의 평균 평점.
    11. **vote_count**: 영화에 대한 평가 개수.
    12. **production_companies**: 영화의 제작 회사 리스트.
    13. **production_countries**: 영화의 제작 국가 리스트.
    14. **spoken_languages**: 영화에서 사용된 언어 리스트.
    15. **cast**: 주요 출연진 리스트.
    16. **crew**: 영화 제작에 참여한 주요 제작진 리스트.
    17. **keywords**: 영화의 키워드 리스트.
    18. **tagline**: 영화의 태그라인(주요 홍보 문구).
    19. **original_language**: 영화의 원어 (예: 영어, 한국어 등).
    20. **homepage**: 영화의 공식 웹사이트 URL.
    21. **poster_path**: 영화 포스터 이미지 URL 경로.

In [16]:
import numpy as np
import pandas as pd

In [17]:
# 데이터 로드
movie_df = pd.read_csv('./data/tmdb_5000_movies.csv')
movie_df.head()
movie_df.shape

(4803, 20)

In [18]:
# 사용할 컬럼 선택
movie_df = movie_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
movie_df.info()
movie_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [ ]:
# 장르 데이터 전처리
from ast import literal_eval

# str -> list(dict)
movie_df['genres'] = movie_df['genres'].apply(literal_eval) # literal_eval은 문자열로 저장된 리스트 또는 딕셔너리를 실제 리스트/딕셔너리로 변환

In [20]:
str_list = '[1, 2, 3]'
lst = literal_eval(str_list)
print(type(lst))

<class 'list'>


In [ ]:
# name value만 꺼내서 list
movie_df['genres'] = movie_df['genres'].apply(lambda genres: [genre['name'] for genre in genres])   # 각 행의 genres 값이 딕셔너리 리스트라면, 각 딕셔너리에서 'name' 키의 값을 추출하여 리스트로 변환
movie_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [ ]:
# list -> str (공백 구분 문자열)
movie_df['genres'] = movie_df['genres'].apply(lambda x: ' '.join(x))    # 각 행에 대해 x(장르 리스트)를 ' '(공백) 기준으로 합침
movie_df['genres']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres, Length: 4803, dtype: object

In [ ]:
# 장르 유사도 측정을 위한 CountVectorizer 사용 
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1, 2))  # 내가 가지고 있는 단어들에 대해서 단어 쌍을 하나, 또는 두개로 조합하겠다 (ab, cd) -> (a, b, c, d, ab, cd)
#  ['나는', '바보가', '아니다'] 에 대해서 ngram_range=(1, 2, 3)  이라면 ['나는', '바보가', '아니다', '나는 바보가', '바보가 아니다', '나는 바보가 아니다'] / 연속된 단어 묶음만 고려
genres_vec = count_vectorizer.fit_transform(movie_df['genres'])
print(genres_vec.shape) # 장르가 276개
print(genres_vec.toarray()[:5]) # 포함되는 장르마다 1이 들어가 있다 
genres_vec_vocab = pd.DataFrame(count_vectorizer.get_feature_names_out())   # 가지고 있는 모든 장르 이름만 가지고 옴 (특성)
genres_vec_vocab

(4803, 276)
[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]]


,0
0,action
1,action adventure
2,action animation
3,action comedy
4,action crime
...,...
271,western drama
272,western history
273,western music
274,western romance


### 🔹 CountVectorizer의 동작 방식  

1. **텍스트를 토큰(단어) 단위로 분리**  
   - 기본적으로 띄어쓰기 기준으로 단어를 나눔.  
   - `tokenizer`를 설정하면 형태소 분석기를 활용할 수도 있음.  

2. **단어 사전(어휘집, Vocabulary) 생성**  
   - 학습 데이터에 있는 **모든 고유한 단어들의 목록**을 만든 후, 각 단어에 인덱스를 부여.  
   - 단어 인덱스는 기본적으로 **알파벳(사전) 순**으로 정렬됨.  

3. **각 문장에서 등장하는 단어 개수(빈도) 세기**  
   - 각 문장을 **단어 등장 횟수 기반의 벡터로 변환**하여 표현.  
   - 단어가 등장하지 않으면 0, 등장하면 해당 횟수를 기록.  


# 🔹 CountVectorizer의 단어 인덱스 배정 원리

1. **중복 단어 제거**
2. **알파벳(사전) 순으로 정렬**
3. **정렬된 순서대로 0부터 인덱스 부여**


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 샘플 문장
corpus = ["나는 바보가 아니다", "너는 바보가 맞다", "우리는 바보가 아니다"]

# CountVectorizer 적용
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# 단어 사전 출력
print("어휘 사전:", vectorizer.vocabulary_)

# 변환된 벡터 출력
print("변환된 벡터:\n", X.toarray())




# {'나는': 0, '너는': 1, '맞다': 2, '바보가': 3, '아니다': 4, '우리는': 5}


# ["나는 바보가 아니다"]    ➝ [1 0 0 1 1 0]  
# ["너는 바보가 맞다"]      ➝ [0 1 1 1 0 0]  
# ["우리는 바보가 아니다"]  ➝ [0 0 0 1 1 1]       -> 각 행은 문장을 의미하고, 각 열은 단어의 등장 횟수를 의미한다



### 코사인 유사도 측정

In [ ]:
# 거리 뿐만 아니라 방향을 따짐